<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Modeling

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook extracts fits predictive biomass growth curves from FIA data


In [ ]:
import numpy as np
import pandas as pd
from forests import load, setup, plot, fit

In [ ]:
setup.plotting()

In [ ]:
df = load.fia(store="local", states="conus")
df = load.terraclim(
    store="local",
    tlim=(2000, 2020),
    vars=["tmax", "ppt"],
    aggs=["mean", "sum"],
    mean=True,
    df=df,
)

In [ ]:
inds = (df["type_code"] == 941) & (df["state"] == "CA")
x = df[inds]["age"]
y = df[inds]["biomass"]
f = [df[inds]["tmax_mean"], df[inds]["ppt_sum"]]
(
    plot.xy(x=x, y=y, color=f[0], cmap="magma", xlim=[0, 200], ylim=[0, 500])
    | plot.xy(x=x, y=y, color=f[1], xlim=[0, 200], ylim=[0, 500])
).resolve_scale(color="independent")

In [ ]:
model = fit.biomass(x=x, y=y, f=f, noise="gamma")

In [ ]:
model.r2(x, f, y)

In [ ]:
xlim = [0, 250]
ylim = [0, 600]
(
    (
        plot.xy(x=x, y=y, color=f[0], cmap="magma", xlim=xlim, ylim=ylim)
        + plot.line(
            x=x,
            y=model.predict(x, f, [90, 50]),
            color=np.nanpercentile(f[0], 90),
            xlim=xlim,
            ylim=ylim,
        )
        + plot.line(
            x=x,
            y=model.predict(x, f, [10, 50]),
            color=np.nanpercentile(f[0], 10),
            xlim=xlim,
            ylim=ylim,
        )
    )
    | (
        plot.xy(x=x, y=y, color=f[1], xlim=xlim, ylim=ylim)
        + plot.line(
            x=x,
            y=model.predict(x, f, [50, 10]),
            color=np.nanpercentile(f[1], 10),
            xlim=xlim,
            ylim=ylim,
        )
        + plot.line(
            x=x,
            y=model.predict(x, f, [50, 90]),
            color=np.nanpercentile(f[1], 90),
            xlim=xlim,
            ylim=ylim,
        )
    )
).resolve_scale(color="independent")

In [ ]:
xlim = [0, 250]
ylim = [0, 600]
(
    (
        plot.xy(x=x, y=y, xlim=xlim, ylim=ylim)
        + plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim)
    )
    | (
        plot.xy(x=x, y=model.sample(x, f), xlim=xlim, ylim=ylim)
        + plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim)
    )
)

In [ ]:
pf = pd.read_parquet("../../scripts/data/biomass.parquet")

In [ ]:
years = np.arange(2020, 2120, 20)
predictions = [pf["BCC-CSM2-MR_ssp245_" + str(year)].mean() for year in years]

In [ ]:
inds = (df["year"] >= 2010) & (df["year"] <= 2018)
(
    plot.xy(
        x=df[inds].groupby("year").mean().index,
        y=df[inds].groupby("year").mean()["biomass"],
        xlim=[2000, 2050],
        ylim=[90, 150],
    )
    + plot.xy(
        x=years, y=predictions, xlim=[2000, 2120], ylim=[90, 160], color="blue"
    )
)

In [ ]:
plot.carto(
    lat=df["lat"],
    lon=df["lon"],
    color=df["biomass"],
    cmap="yellowgreen",
    clim=(0, 250),
)

In [ ]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["BCC-CSM2-MR_ssp245_2020"],
    cmap="yellowgreen",
    clim=(0, 250),
)

In [ ]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["2100"],
    cmap="yellowgreen",
    clim=(0, 250),
)

In [ ]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["2100"] - pf["2020"],
    cmap="yellowgreen",
    clim=(0, 100),
)

In [ ]:
import rasterio

In [ ]:
r = rasterio.open(
    "/Users/freeman/workdir/carbonplan-data/processed/nlcd/conus/4000m/2016_c42.tif"
)

In [ ]:
a = r.read(1)

In [ ]:
from showit import image
%matplotlib inline

In [ ]:
a.shape

In [ ]:
image(a, size=12)

In [ ]:
from rasterio import Affine
from rasterio.transform import rowcol
from pyproj import transform, Proj
from carbonplan_data.utils import albers_conus_crs, albers_conus_transform

In [ ]:
inds = df["year"] > 2010
df_sub = df[inds].reset_index()
t = Affine(*albers_conus_transform(4000))
p1 = Proj(albers_conus_crs())
p2 = Proj(proj="latlong", datum="WGS84")
x, y = transform(p2, p1, df_sub["lon"].values, df_sub["lat"].values)
rc = rowcol(t, x, y)

In [ ]:
m = np.zeros(a.shape)
b = np.zeros(a.shape)

In [ ]:
for i, (r, c) in enumerate(zip(rc[0], rc[1])):
    m[r, c] += 1
    b[r, c] += df_sub["biomass"][i]

In [ ]:
m[rc[0], rc[1]] = df["biomass"]

In [ ]:
image(a[0:300, 0:300], size=12)

In [ ]:
image(b[0:300, 0:300] / m[0:300, 0:300], clim=(0, 400), size=12)

In [ ]:
image(m[0:300, 0:300], clim=(0, 1), size=12)

In [ ]:
image(m, clim=(0, 200), cmap="viridis", size=12)

In [ ]:
image((m[0:400, 0:400] != 201) & (m[0:400, 0:400] != 0), size=12)

In [ ]:
np.unique(a[0:400, 0:400])

In [ ]:
image(
    ((m[0:400, 0:400] != 201) & (m[0:400, 0:400] != 0))
    + 0.5 * (a[0:400, 0:400] == 301),
    size=12,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sum(types == df["type_code"]) / len(types)

In [ ]:
plot.xy(x=types, y=df["type_code"])

In [ ]:
image(m[0:300, 0:300], size=12)